In [48]:
import time
import pandas as pd
import numpy as np
from datetime import datetime
import re
import locale
import pyautogui

from utils import *

# Importing Selenium library and relevant classes
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains

Define: ```Elemento a buscar``` ```Estado``` ```distancia``` ```n_scrolls_cada_vez```
 ```Lista a incluir``` 
 ```Lista a excluir```

In [49]:
elemento_a_buscar = 'iphone 16'
n_iphone = elemento_a_buscar.split()[1] # para otros elementos modificlar el bucle que verifica si el item es correcto

#funcion para seleccionar los articulos que quiero:
# si una la tiene o no la tiene se incluye o excluye.
# por ejemplo si buscadas = 15 y 16
#                excluyentes = funda cristal
#                'funda iphone 15' se excluye
#                'iphone'      no se incluye porque no tiene el 15 ni el 16
cadenas_buscadas = (
                    n_iphone,
                    n_iphone + ','
                    )  # la coma es para que python lo vea como una tupla
cadenas_excluyentes = (
                    "funda",
                    "fundas",
                    "carcasa",
                    "carcasas",
                    "protector",
                    "cristal",
                    "4", "5", "6",
                    "4s", "5s", "6s"
                    "16gb",
                    "caja"
)

# 'new', 'as_good_as_new', 'good'
estado='good'
# para poner varios escribir separando con coma y sin espacios asi: 'as_good_as_new,good'
distancia = '30' # distancia en km

#n_scrolls_total = 40 # no se usa, numero de veces que haremos scroll abajo. Cada 10 carga datos
n_scrolls_cada_vez = 15 
if type(cadenas_buscadas) is not tuple:
    print("cadenas_buscadas no es tupla")
if type(estado) is not str:
    print("estado no es str")  

In [50]:
#Abre la venana
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

ele_1 = elemento_a_buscar.split()[0]
ele_2 = elemento_a_buscar.split()[1]
#driver.get('https://es.wallapop.com');
# para Madrid no cambiar las coordenadas
url = 'https://es.wallapop.com/app/search?filters_source=quick_filters&keywords=' + ele_1 + '%20' + ele_2 + '&longitude=-3.69196&latitude=40.41956&distance='+ str(distancia) + '000&condition=' + str(estado)
driver.get(url)
print(url)
time.sleep(1)
driver.maximize_window()  
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(1)

https://es.wallapop.com/app/search?filters_source=quick_filters&keywords=iphone%2016&longitude=-3.69196&latitude=40.41956&distance=30000&condition=good


In [51]:
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()


In [52]:
time.sleep(8)
# clico 3 veces fuera del cuadrado para saltar el tutorial
x_coord = 1600  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(2)
x_coord = 1600  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(2)
x_coord = 1600  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(4)

In [53]:
# no se que hace. creo que cargar 40 articulos más
# button = WebDriverWait(driver, 6).until(
#     EC.element_to_be_clickable((By.XPATH, "//walla-button[contains(@class, 'w-100')]"))
# )
# # Haz clic en el botón
# button.click()
# time.sleep(2)

In [54]:
# Clicamos para ver más
time.sleep(3)
boton_ver_mas = driver.find_element(By.ID, "btn-load-more")
boton_ver_mas.click()
time.sleep(2)

In [55]:
# Creación de la tabla con las columnas que buscamos
try:
    del df
except NameError:
    pass
columnas = ['id', 'nombre', 'precio', 'estado', 'reservado', 'url']
df = pd.DataFrame(columns=columnas)

In [56]:
time.sleep(2)
x_coord = 1600
y_coord = 780 
pyautogui.click(x=x_coord, y=y_coord)

# Scrolling

In [57]:
# Clicamos para ver más y hacemos scroll down n veces
n = 0
while n < n_scrolls_cada_vez:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    n = n + 1

In [58]:
def item_reservado(elem):
    # Asumiendo que la clase 'reservado' en algún elemento interno indica que el artículo está reservado
    try:
        elemento_reservado = elem.find_element(By.CSS_SELECTOR, ".clase_indicativa_de_reservado")
        return True
    except:
        return False

In [59]:
print(is_iphone_xx('iphone 16 de 128gb ultramarine', '16', cadenas_buscadas, cadenas_excluyentes))

True


In [60]:
# Hasta aqui tarda 
# Extraigo los datos y los añado en la tabla. 
n_iphone = '16'
elementos = driver.find_elements(By.CSS_SELECTOR, "a.ItemCardList__item")
print("Número de elementos encontrados:", len(elementos))
for index, elem in enumerate(elementos):
    print(index)
    url_articulo = elem.get_attribute('href')
    id_articulo = url_articulo.split('-')[-1]
    if not ya_existe_articulo(id_articulo, df):
        nombre = elem.get_attribute('title')
        nombre = nombre.lower()
        if is_iphone_xx(nombre, n_iphone, cadenas_buscadas, cadenas_excluyentes):
            precio = elem.find_element(By.CSS_SELECTOR, ".ItemCard__price").text.strip()
            precio = precio.split(' ')[0]
            reservado = item_reservado(elem)  
            # Guarda en la tabla
            print("---guardado---" + nombre)
            df.loc[index] = [id_articulo, nombre, precio, estado, reservado, url_articulo]
        else:   
            print("### excluido ######", nombre)
    else:
        print(    "#ya existe con id: ", nombre, id_articulo)
        
    
df = df.reset_index(drop=True)
print(df)

Número de elementos encontrados: 189
0
### excluido ###### iphone se gsm+cdma 16gb gold
1
### excluido ###### iphone se
2
---guardado---iphone 16 128 gb, rosa a estrenar
3
### excluido ###### iphone 11
4
---guardado---iphone 16 plus 128gb ultramarino 100% bateria
5
### excluido ###### iphone 5s
6
### excluido ###### iphone xr 128gb negro 100% bateria
7
### excluido ###### iphone 13 128gb negro 100% bateria 
8
### excluido ###### iphone 13 pro max 128gb verde 
9
### excluido ###### iphone 12 pro 256gb grafito 
10
### excluido ###### iphone 15 plus 128gb negro 100% bateria
11
### excluido ###### iphone 14 128gb amarillo 100% bateria
12
### excluido ###### iphone 11 64gb rojo 100% bateria 
13
### excluido ###### iphone xs 256gb negro 100% bateria
14
### excluido ###### iphone se 2020 128gb negro 92% bateria
15
### excluido ###### placa base iphone 3g 16gb
16
### excluido ###### iphone 11 pro max 256gb verde 100% bateria 
17
### excluido ###### iphone 15 pro 256gb natural titanium 100% bat

In [61]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df

,id,nombre,precio,estado,reservado,url
0,1068680575,"iphone 16 128 gb, rosa a estrenar","803,95",good,False,https://es.wallapop.com/item/iphone-16-128-gb-rosa-a-estrenar-1068680575
1,1068638513,iphone 16 plus 128gb ultramarino 100% bateria,"949,00",good,False,https://es.wallapop.com/item/iphone-16-plus-128gb-ultramarino-100-bateria-1068638513
2,1068402997,iphone 6 silver 16 gb,"70,00",good,False,https://es.wallapop.com/item/iphone-6-silver-16-gb-1068402997
3,1063493016,iphone 16,"810,00",good,False,https://es.wallapop.com/item/iphone-16-1063493016
4,1066517058,iphone 5s 16 gb,"50,00",good,False,https://es.wallapop.com/item/iphone-5s-16-gb-1066517058
5,1063908118,iphone 16 128gb lila,"799,99",good,False,https://es.wallapop.com/item/iphone-16-128gb-lila-1063908118
6,1060834519,iphone 16,"830,00",good,False,https://es.wallapop.com/item/iphone-16-1060834519
7,1062639161,"teléfono apple iphone 3, 16 gb","15,00",good,False,https://es.wallapop.com/item/telefono-apple-iphone-3-16-gb-1062639161
8,1055322025,iphone 16,"895,00",good,False,https://es.wallapop.com/item/iphone-16-1055322025
9,1051268843,telefono iphone 3gs 16 gb,"60,00",good,False,https://es.wallapop.com/item/telefono-iphone-3gs-16-gb-1051268843


In [62]:
hoy_formateada = datetime.now().strftime("%Y%m%d")
print(hoy_formateada)
print(elemento_a_buscar)
print(estado)

20241112
iphone 16
good


In [63]:
nombre_archivo_pkl = hoy_formateada + '_' + elemento_a_buscar + '_' + estado
df.to_csv('datos_raw/' + nombre_archivo_pkl + '.csv')

In [64]:
try:
    df = pd.read_csv('datos_raw/' + nombre_archivo_pkl + '.csv')
    print("pickel guardado")
except:
    print("Error leyendo archivo! puede que no se haya guardado bien")
#driver.quit()

pickel guardado
